In [1]:
import keras
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score

class_names = ["confirmed", "crossedout", "empty"]

2024-09-14 09:40:39.645163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 09:40:39.662151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 09:40:39.666939: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 09:40:39.679277: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-14 09:40:41.403608: W tensorflow/compiler/tf2

In [2]:
model = keras.saving.load_model("models/omr_model.keras")
test_ds =  tf.data.Dataset.load("datasets/ds_test").rebatch(256)

I0000 00:00:1726317645.113410   15760 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726317645.153347   15760 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726317645.153549   15760 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726317645.154393   15760 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

## Métricas de desempenho

Algumas métricas de desemprenho para o modelo são calculadas neste notebook.

Para isso, será utilizado o modulo `metrics` do pacote `scikit-learn`.

In [3]:
true_labels = []
for _, label in test_ds:
    true_labels.append(label.numpy())

true_labels = np.concatenate(true_labels)

2024-09-14 09:40:50.164994: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
results = model.predict(test_ds)

I0000 00:00:1726317651.116027   15808 service.cc:146] XLA service 0x79082c002630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726317651.116044   15808 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2024-09-14 09:40:51.151038: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-14 09:40:51.504523: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


      5/Unknown 7s 46ms/step

I0000 00:00:1726317657.368622   15808 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 131ms/step


2024-09-14 09:41:04.205086: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


In [5]:
predicted_labels = np.argmax(results, axis=1)

In [6]:
_, predicted_ammount = np.unique(predicted_labels, return_counts=True)

In [7]:
precision, recall, fscore, support = precision_recall_fscore_support(true_labels, predicted_labels, average=None)

In [8]:
auc = roc_auc_score(true_labels, results, average=None, multi_class='ovr')

Inicialmente, olharemos para a acurácia.

Para o modelo treinado, obtemos uma acurácia de 99,58%, que é extremamente alta.

Entretanto, vale lembrar que o modelo possui categoriais desbalanceadas, principalmente, a categoria "crossedout".

Portanto, pode ser interessante avaliar outras métricas.

In [9]:
acc = accuracy_score(true_labels, predicted_labels) * 100

print(f"Test dataset accuracy: {acc:.2f}%")

Test dataset accuracy: 99.58%


A tabela a seguir exibe um resumo de algumas métricas, abertas por classe.

Podemos ver que apesar do desblanceamento, o modelo possui métricas aceitáveis para todas as classes.

Para a classe "crossedout", observamos apenas 80 ocorrências nos dados de teste, enquanto que a classe "confirmed" possui mais de 4000 e a classe "empty" quase 9000.

Entretanto, até mesmo para a classe crossedout, as medidas como recall e precisão superam 70%.

In [10]:
import pandas as pd

pd.DataFrame(
    {"class": class_names, "support": support, "predicted_ammount": predicted_ammount, "precision": precision, "recall": recall, "fscore": fscore, "auc": auc}
)

,class,support,predicted_ammount,precision,recall,fscore,auc
0,confirmed,4463,4460,0.995291,0.994622,0.994957,0.999751
1,crossedout,80,81,0.703704,0.712500,0.708075,0.987857
2,empty,8876,8878,0.998648,0.998873,0.998761,0.999972


Na matriz de confusão, podemos ver que a grande maioria dos erros ocorrem na classe "crossedout".

As caixas vazias e confirmadas raramente são classificadas incorretamente.

Ainda assim, mesmo a classe "crossedout" tem uma taxa de acerto relevante.

In [11]:
cm = confusion_matrix(true_labels, predicted_labels)

df = pd.DataFrame(cm, index=class_names, columns=class_names)
df.index.name = "True"
df.columns.name = "Predicted"
df

Predicted,confirmed,crossedout,empty
True,,,
confirmed,4439,16,8
crossedout,19,57,4
empty,2,8,8866


Portanto, o modelo parece ser bastante eficaz para a tarefa de classificação.

No notebook `predict.ipynb`, é possível ver algumas das observações e suas previsões.

De fato, a classe crossedout parece possuir algumas marcações incorretas nos dados, ou algumas marcações que de fato não são tão claras de diferenciar de uma caixa confirmada, por exemplo.

Por esse motivo, a aplicação onde o modelo será utilizado necessitará de um processo de validação, onde os usuários poderão corrigir as previsões incorretas.

Além disso, o modelo prevê a classe para cada caixa do teste, o que pode ser um pouco incoveniente para o usuário, que precisa informar onde as caixas estão localizadas.

Por esse motivo, o foco inicial da aplicação será em gerar os dados da localização de cada caixa, além do label associado, para que no futuro seja treinado um modelo de segmentação de imagem que além de classificar cada caixa, também detectará sua localização na imagem.

De qualquer forma, o modelo em sua forma atual pode ser utilizado, em conjunto com ferramentas como o *template matching* do Open-CV para identificar as respostas dos testes preenchidos de forma semi-automática.